In [86]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

from google.colab import drive 
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [100]:
# Path to data
data_dir  = '/content/drive/MyDrive/Masters/Semester 2/Data Mining & Visualisation/Assessments/Assessment1/Code/dataset/'

# Get the path to the normal and pneumonia sub-directories
normal_files_dir = data_dir + 'Normal'        # Locate the Normal Images folder
cov19_files_dir = data_dir + 'Covid-19'       # Locate the COVID-19 Image folder

train_path = data_dir + 'Train'
valid_path = data_dir + 'Test'

# Get the list of all the images
normal_files = glob(normal_files_dir+'/*.*')      
cov19_files = glob(cov19_files_dir+'/*.*')

print('Length of covid-19 file is ', len(cov19_files))     # Print the length of the total images
print('Length of normal lungs file is ',len(normal_files))    # Print the length of the normal images

#  Split the Images data into a 80% (Train) / 20% (Test) process.
cov19_train = np.random.choice(cov19_files, size=80, replace=False).tolist() 
normal_train = np.random.choice(normal_files, size=80, replace=False).tolist() 
print('Length of covid-19 training set is ',len(cov19_train))
print('Length of normal lungs training set is ',len(normal_train))

cov19_test = list(set(cov19_files)-set(cov19_train))            # Load only 20 %
normal_test = list(set(normal_files)-set(normal_train))         # Load only 20 %

print('Length of covid-19 testing set is ',len(cov19_test))
print('Length of normal lungs testing set is ',len(normal_test))




Length of covid-19 file is  100
Length of normal lungs file is  100
Length of covid-19 training set is  80
Length of normal lungs training set is  80
Length of covid-19 testing set is  20
Length of normal lungs testing set is  20


In [91]:
import shutil, random
i = 0

for filename in random.sample(normal_files, 80):
    i+=1
    shutil.copy(filename, train_path+'/Normal'+ str(i))

for filename in random.sample(covid_19_files, 80):
    i+=1
    shutil.copy(filename, train_path+'/Covid-19'+ str(i))

In [92]:
i = 0
# Copy the data from the file for Testing Sets
for filename in random.sample(normal_files, 20):
    i+=1
    shutil.copy(filename, valid_path+'/Normal'+ str(i))

for filename in random.sample(covid_19_files, 20):
    i+=1
    shutil.copy(filename, valid_path+'/covid_19'+ str(i))

['/content/drive/MyDrive/Masters/Semester 2/Data Mining & Visualisation/Assessments/Assessment1/Code/dataset/Normal/NORMAL2-IM-0890-0001.jpeg']


In [ ]:
# re-size all the images to this
IMAGE_SIZE = [224, 224]

# add preprocessing layer to the front of ResNet50
resnet = ResNet50(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

# don't train existing weights - # We freeze the weights of the model by setting trainable as “False”
for layer in resnet.layers:
  layer.trainable = False
    
  # useful for getting number of classes
folders = glob(train_path+'/*')
  
# our layers - you can add more if you want
x = Flatten()(resnet.output)
# x = Dense(1000, activation='relu')(x)
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=resnet.input, outputs=prediction)

# view the structure of the model
model.summary()

# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

#  Apply Data Augmentation to Images 
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)



training_set = train_datagen.flow_from_directory(directory = train_path,
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory(directory = valid_path,
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

'''r=model.fit_generator(training_set,
                         samples_per_epoch = 8000,
                         nb_epoch = 5,
                         validation_data = test_set,
                         nb_val_samples = 2000)'''
print(len(training_set))
print(len(test_set))


# fit the model
model_fitting = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=100,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)


In [ ]:
plt.plot(model_fitting.history['loss'], label='train loss')
plt.plot(model_fitting.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')



In [ ]:
plt.plot(model_fitting.history['accuracy'], label='train acc')
plt.plot(model_fitting.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('Accuracy')
